In [ ]:
import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], '..'))
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from ephys import *
from ephys_utils import select_spikes_by_trial, transform_spike_data, find_template_for_clusters
from spatial_analysis import *

from session_utils import *

session_list = find_all_sessions(sheet_path = 'https://docs.google.com/spreadsheets/d/1_Xs5i-rHNTywV-WuQ8-TZliSjTxQCCqGWOD2AL_LIq0/edit#gid=0',
                                 data_path = '/home/isabella/Documents/isabella/jake/recording_data',
                                 sorting_suffix = 'sorting_ks2_custom')

total_clusters = 0

for path_to_session in session_list.values():

    # Load spike and position data
    obj = ephys(recording_type = 'nexus', path = path_to_session)

    obj.load_spikes('good')
    obj.load_pos([i for i, s in enumerate(obj.trial_list)], output_flag = False)
    pos_sample_rate = obj.pos_data[0]['pos_sampling_rate']
    
    # Filter based on metrics
    cluster_info = obj.spike_data['cluster_info'] #Get cluster info from phy

    ## Get cluster depths and exclude any outside of 0 +-200um
    cluster_info = cluster_info[cluster_info['depth'].between(-200, 200)]

    ## Filter for mean firing rate - CHECK EXACT VALUES
    cluster_info = cluster_info[cluster_info['fr'].between(1, 10)]

    ## Filter for spike width from template
    # Find kilosort template for each cluster (closely approximates spike width) and add to dataframe
    template_per_cluster = find_template_for_clusters(obj.spike_data['spike_clusters'], obj.spike_data['spike_templates'])
    template_df = pd.DataFrame.from_dict(template_per_cluster, orient = 'index')
    template_df.columns = ['template_id']

    cluster_info = cluster_info.join(template_df, how = 'left')
    
    if not cluster_info.empty:
        # Load templates.npy
        templates = np.load(f'{path_to_session}/{path_to_session[-8:-6]}{path_to_session[-5:-3]}{path_to_session[-2:]}_sorting_ks2_custom/templates.npy')
        # Load inverse whitening matrix and apply to unwhiten templates
        whitening_matrix_inv =  np.load(f'{path_to_session}/{path_to_session[-8:-6]}{path_to_session[-5:-3]}{path_to_session[-2:]}_sorting_ks2_custom/whitening_mat_inv.npy')
        unwhitened_templates = np.einsum('ijk,kl->ijl', templates, whitening_matrix_inv)


        # Add template values to dataframe
        cluster_info['template'] = cluster_info.apply(lambda row: templates[int(row['template_id']), :, int(row['ch'])], axis=1)
    
    if not cluster_info.empty:
        # Work out template width peak to trough
        sampling_rate = obj.spike_data['sampling_rate']

        spike_width_samples = cluster_info['template'].apply(
            lambda x: np.abs(np.argmax(x) - np.argmin(x))
        )

        # Convert spike width to microseconds and add to dataframe
        spike_width_microseconds = (spike_width_samples/sampling_rate)*1000000
        cluster_info['spike_width_microseconds'] = spike_width_microseconds

        # Filter for spike width > 300us as in Wills et al., 2010
        cluster_info = cluster_info[cluster_info['spike_width_microseconds'] > 300]

    # Reload spike data only for included cells
    clusters_inc = list(cluster_info.index)
    obj.load_spikes(clusters_to_load = clusters_inc)
    
    clusters_inc = cluster_info.index
    n_clusters_inc = len(cluster_info.index)

    np.save(f'{path_to_session}/clusters_inc.npy', clusters_inc)
    print(f'{path_to_session}:   {n_clusters_inc} clusters saved as candidate pyramidal cells. Cluster IDs: {clusters_inc.values}')
    total_clusters += n_clusters_inc
    

204 LED swaps detected and fixed


/data/isabella/jake/ephys_analysis/pyScan/postprocessing/../postprocessing/postprocess_pos_data.py:219: RuntimeWarning: invalid value encountered in remainder
  dir_disp = np.mod(np.arctan2(-pos.iloc[1, 1:].values + pos.iloc[1, :-1].values, pos.iloc[0, 1:].values - pos.iloc[0, :-1].values) * 180 / np.pi, 360)


/home/isabella/Documents/isabella/jake/recording_data/r1354/2023-05-03:
0 clusters saved as candidate pyramidal cells. Cluster IDs: []
167 LED swaps detected and fixed
233 LED swaps detected and fixed
322 LED swaps detected and fixed
/home/isabella/Documents/isabella/jake/recording_data/r1354/2023-05-04:
1 clusters saved as candidate pyramidal cells. Cluster IDs: [229]
431 LED swaps detected and fixed
244 LED swaps detected and fixed
272 LED swaps detected and fixed
220 LED swaps detected and fixed
/home/isabella/Documents/isabella/jake/recording_data/r1354/2023-05-05:
0 clusters saved as candidate pyramidal cells. Cluster IDs: []
441 LED swaps detected and fixed
202 LED swaps detected and fixed
64 LED swaps detected and fixed
25 LED swaps detected and fixed
/home/isabella/Documents/isabella/jake/recording_data/r1354/2023-05-06:
9 clusters saved as candidate pyramidal cells. Cluster IDs: [226 236 242 247 251 260 264 281 308]
248 LED swaps detected and fixed
330 LED swaps detected and f

FileNotFoundError: [Errno 2] No such file or directory: '/home/isabella/Documents/isabella/jake/recording_data/r1398/2023-08-03/230803_sorting_ks2_custom/cluster_info.tsv'

SyntaxError: can't assign to lambda (640551095.py, line 1)